    TODO Instead of using the tiles based ont his file, I should use tiles from my target grid, the CLC 100m.
    
    TODO Water should be 0 not, nodata
    TODO vrt for the smaller tiles. 

With pkcomposite -cr mean i can maybe do it, although this is ment to calculate the mean between 2 layers. 
so maybe not. Lateron I can use it to combine all my maps tho, and include weights. 

Here we take the 10m CLC map turn it into forest/nonforest and downsample it to the 100m map

recode to forest/nonforest/NA
First lets create a list of forest classes
Note that we want to classify the nodata in water 254 also as "noforest", so we then correctly downlsample with mean 

In [9]:
%%bash
for((i=1;i<=11;++i));do
  if(($i >=2 && $i <=4));then
    echo "$i 1"
  else
    echo "$i 0";fi
done > /tmp/recode4bash.txt 

echo "254 0" >> /tmp/recode4bash.txt 
echo "255 0" >> /tmp/recode4bash.txt 

In [10]:
!head -5 /tmp/recode4bash.txt 
!echo ------------------
!tail -5 /tmp/recode4bash.txt 

1 0
2 1
3 1
4 1
5 0
------------------
9 0
10 0
11 0
254 0
255 0


Get the extent and resolution from the 100m map and use that to calculate the mean

In [22]:
%%bash
i=3
j=3

blockfile=out_data/CLC10m/blocks16/CLC10_$i$j.tif
outfile=out_data/CLC10m/blocks16/recode_CLC10_$i$j.tif
echo $((  (i-1)*162500   ))
echo $((  (j-1)*115000   )) 



325000
230000


Split it into chunks
-srcwin Selects a subwindow from the source image for copying based on pixel/line location.


In [20]:
%%bash
start=`date +%s`

cat /tmp/recode4bash.txt

for i in {1..4..1} ;do
for j in {1..4..1} ;do


blockfile=out_data/CLC10m/blocks16/CLC10_$i$j.tif
outfile=out_data/CLC10m/blocks16/recode_CLC10_$i$j.tif

xstart=$((  (i-1)*162500   ))
ystart=$((  (j-1)*115000   )) 

# gdal_translate -co COMPRESS=DEFLATE -co ZLEVEL=9 -srcwin $xstart $ystart 162500 115000 in_data/CLC/CLC_2018_10m/Data/CLMS_CLCplus_RASTER_2018_010m_eu_03035_V1_1.tif $blockfile

pkreclass -co COMPRESS=DEFLATE -co ZLEVEL=9  \
-code /tmp/recode4bash.txt                   \
-nodata 48                                   \
-ot Float64                                  \
-co BIGTIFF=YES                              \
-i $blockfile                                \
-o $outfile                                  \



done
done
end=`date +%s`

expr $end - $start

1 0
2 1
3 1
4 1
5 0
6 0
7 0
8 0
9 0
10 0
11 0
254 0
255 0
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50.

In [58]:
!gdalinfo -hist out_data/CLC10m/blocks16/recode_CLC10_33.tif > out_data/CLC10m/blocks16/gdalhist_recode_CLC10_33.txt

In [28]:
!cat out_data/CLC10m/blocks16/gdalhist_recode_CLC10_33.txt

Driver: GTiff/GeoTIFF
Files: /media/sf_geocomputation_class/out_data/CLC10m/blocks16/recode_CLC10_33.tif
Size is 162500, 115000
Coordinate System is:
PROJCRS["ETRS89-extended / LAEA Europe",
    BASEGEOGCRS["ETRS89",
        ENSEMBLE["European Terrestrial Reference System 1989 ensemble",
            MEMBER["European Terrestrial Reference Frame 1989"],
            MEMBER["European Terrestrial Reference Frame 1990"],
            MEMBER["European Terrestrial Reference Frame 1991"],
            MEMBER["European Terrestrial Reference Frame 1992"],
            MEMBER["European Terrestrial Reference Frame 1993"],
            MEMBER["European Terrestrial Reference Frame 1994"],
            MEMBER["European Terrestrial Reference Frame 1996"],
            MEMBER["European Terrestrial Reference Frame 1997"],
            MEMBER["European Terrestrial Reference Frame 2000"],
            MEMBER["European Terrestrial Reference Frame 2005"],
            MEMBER["European Terrestrial Reference Frame 2014

In [29]:
!gdallocationinfo out_data/CLC10m/blocks16/recode_CLC10_33.tif 50 50
!echo ------------------------
!gdalinfo out_data/CLC10m/blocks16/recode_CLC10_33.tif |grep Size

Report:
  Location: (50P,50L)
  Band 1:
    Value: 1
------------------------
Size is 162500, 115000
Pixel Size = (10.000000000000000,-10.000000000000000)


Now upsampling we might want to do this using a vrt over all files at once to avoid edge effects

Make a VRT of all the recodes

In [1]:
!gdalbuildvrt -overwrite out_data/CLC10m/recode_CLC10.vrt out_data/CLC10m/blocks16/recode_CLC10_??.tif

0...10...20...30...40...50...60...70...80...90...100 - done.


In [2]:
%%bash
infile=out_data/CLC10m/recode_CLC10.vrt
outfile=out_data/CLC10m/mean100m_CLC10.tif


start=`date +%s`
gdal_translate -ot Float64  \
-tr 100 100                 \
-r average                  \
$infile  $outfile           \

end=`date +%s`

expr $end - $start

Input file size is 650000, 460000
0...10...20...30...40...50...60...70...80...90...100 - done.
1422


In [ ]:
take the mode 

In [ ]:
%%bash
infile=out_data/CLC10m/recode_CLC10.vrt
outfile=out_data/CLC10m/mode100m_CLC10.tif


start=`date +%s`
gdal_translate -ot Float64  \
-tr 100 100                 \
-r mode                     \
$infile  $outfile           \

end=`date +%s`

expr $end - $start

calculate the sd min and max of the superpixels probably with pkfilter since the grids are aligned already

In [3]:
!wc -c out_data/CLC10m/mean100m_CLC10.tif


23920552526 out_data/CLC10m/mean100m_CLC10.tif
